In [1]:
import pandas as pd
from src.services.cases import get_cases, get_single_case
from src.models.cases import Case
from collections.abc import MutableMapping
import datetime

/Users/aennassiri/Projects/Personal/ticket-washer/src/core/config.py
ROOT_PATH: /Users/aennassiri/Projects/Personal/ticket-washer
ROOT_PATH: /Users/aennassiri/Projects/Personal/ticket-washer


## Data Loading

In [2]:
cases = get_cases(
    start_date=datetime.datetime(2023, 10, 1), 
    end_date=datetime.datetime(2023, 11, 9)
)

/Users/aennassiri/Projects/Personal/ticket-washer/venv/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/Users/aennassiri/Projects/Personal/ticket-washer/src/services/cases.py:35: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  cases_list = cases_list.where(


In [4]:
cases_deep_copy = [case.model_copy() for case in cases]

# cases = [case.model_copy() for case in cases_deep_copy]

In [5]:

def flatten(dictionary, parent_key="", separator="_"):
    items = []
    for key, value in dictionary.items():
        new_key = parent_key + separator + key if parent_key else key
        if isinstance(value, MutableMapping):
            items.extend(flatten(value, new_key, separator=separator).items())
        else:
            items.append((new_key, value))
    return dict(items)


In [6]:
def get_case_dict(case: Case):
    court_date = None
    court_time = None
    case_data = flatten(case.model_dump())

    # Transform case location
    location = case.location
    if location is None:
        location = ""

    court_desc = case.court_desc
    if court_desc is None:
        court_desc = ""

    if "municipal" in court_desc.lower() or "municipal" in location.lower():
        case_data["city"] = location.lower().replace("municipal", "").upper()
        case_data["location"] = "MUNICIPAL"
    elif "circuit" in court_desc.lower() or "circuit" in location.lower():
        case_data["city"] = location.lower().replace("circuit", "").upper()
        if "county" not in case_data["city"].lower():
            case_data["city"] += " COUNTY"
        case_data["location"] = "CIRCUIT"
    
    case_data["city"] = case_data.get("city", "").replace("COURT", "")
    

    # Adding the current date short
    case_data["current_date_short"] = (
        datetime.datetime.now().strftime("%B %d, %Y").upper()
    )

    # Adding charges 
    charges = case_data.get("charges", [{"charge_description": ""}])
    if charges:
        case_data["charges_description"] = charges[0].get("charge_description", "")
    else: 
        case_data["charges_description"] = ""

    # Adding the court date and time
    emails = []

    if case.dockets is not None:
        for docket in case.dockets:
            if docket.get("enotice") is not None: 
                    print("enotic in here")
                    emails = [email.get("email") for email in docket.get("enotice", [])]
            if "initial" in docket.get("docket_desc", "").lower():
                # Get the associated_docketscheduledinfo
                schedule = docket.get("associated_docketscheduledinfo", {})
                if isinstance(schedule, list) and len(schedule) > 0:
                    schedule = schedule.pop()
                else:
                    schedule = {}
                court_date = schedule.get("associated_date", "")
                court_time = schedule.get("associated_time", "")
                break

    judge = None

    if case.judge is not None:
        middle_name = case.judge.get("middle_name", None)
        if middle_name is None:
            judge = f"{case.judge.get('first_name', '')} {case.judge.get('last_name', '')}"
        else:
            judge = f"{case.judge.get('first_name', '')} {middle_name} {case.judge.get('last_name', '')}"


    output = {
        "case_id": case.case_id,
        "first_name": case.first_name,
        "last_name": case.last_name,
        "middle_name": case.middle_name,
        "location": case.location,
        "birth_date": case.birth_date,
        "court_date": court_date,
        "court_time": court_time,
        "judge": judge,
        "current_date": case_data.get("current_date_short", ""),
        "city": case_data.get("city", ""),
        "location": case_data.get("location", ""),
        "location_original": location,
        "charges_description": case_data.get("charges_description", ""),
        "emails": emails
    }
    return output

In [7]:
cases_df = pd.DataFrame([get_case_dict(c) for c in cases])

In [8]:
cases_df.judge.value_counts()

judge
GARRY HELM              1215
KEVIN KELLY             1062
WENDY GARRISON          1011
WILLIAM BUCHHOLZ         805
EVAN SULLIVAN            753
                        ... 
JOEL MILLER                1
JENNIFER FISHER            1
CHRISTOPHER STINNETT       1
BENJAMIN THOMPSON          1
ANDREW BRISCOE             1
Name: count, Length: 296, dtype: int64

In [16]:
cases_df.drop(columns="emails", inplace=True)

In [17]:
cases_df.groupby(["location_original","judge", "charges_description"]).case_id.count().sort_values(ascending=False)

location_original           judge           charges_description                                                              
Springfield Municipal       WENDY GARRISON  Parking Violation                                                                    293
St Louis Co Municipal       EVAN SULLIVAN   Oper Veh Without Maintaining Financial Responsibility                                270
Springfield Municipal       WENDY GARRISON  Exceeded Posted Speed Limit (11-15 Mph Over)                                         202
Maryland Heights Municipal  KEVIN KELLY     Oper Veh Without Maintaining Financial Responsibility                                191
                                            Exceeded Posted Speed Limit (20-25 Mph Over)                                         187
                                                                                                                                ... 
Liberty Municipal           THOMAS CAPPS    Speeding (26-30 Mph Over)       

In [52]:
cases_df.charges_description.value_counts()

exclude = [
    "expired plates",
    "oper veh without",
    "parking",
    "seat belt",
    "failed to register",
    "failed to register vehicle",
    "failure to register motor vehicle",
    "stealing/larceny/theft",
    "trespass",
    "vehicle license/inspection/title",
    "nuisance violation",
    "driver of mtr veh failed to wear properly adjusted/ fastened safety belt",
    "assault",
    "resisting arrest",
    "shoplifting",
    "defective equipment",
    "animal at large",
    "equipment violation",
    "possess drug paraphernalia",
    "domestic assault",
]

In [55]:
charges_count = cases_df[~cases_df.charges_description.map(lambda x: x.lower()).isin(exclude)].charges_description.value_counts()

In [57]:
charges_count.head(30)

charges_description
Exceeded Posted Speed Limit (11-15 Mph Over)                                                                           1793
Exceeded Posted Speed Limit (20-25 Mph Over)                                                                           1611
Exceeded Posted Speed Limit (16-19 Mph Over)                                                                           1543
Exceeded Posted Speed Limit (Exceeded By 11 - 15 Mph)                                                                  1473
Exceeded Posted Speed Limit (Exceeded By 16 - 19 Mph)                                                                  1400
Exceeded Posted Speed Limit (Exceeded By 20 - 25  Mph)                                                                 1390
                                                                                                                        974
Driving While Revoked Or Suspended                                                                              

In [67]:
charges_to_filter = charges_count[
    (charges_count > 112)
 ].index

In [72]:
cases_df[cases_df.charges_description.isin(charges_to_filter)].groupby(
    ["location_original", "judge", "charges_description"]
).case_id.count().sort_values(ascending=False).to_csv("work_rules.csv")

In [136]:
cases_df.describe()

,case_id,first_name,last_name,middle_name,location,birth_date,court_date,court_time,judge,current_date,city,location_original,charges_description,emails
count,9302,9265,9302,8111,9113,9011,9030,9030,9059,9302,9302,9302,9302,9302
unique,9302,2784,4935,1864,2,6045,76,32,240,1,305,305,247,1
top,120266318,MICHAEL,SMITH,MARIE,MUNICIPAL,01/01/0001,11/01/2023,9:00 AM,GARRY HELM,"SEPTEMBER 29, 2023",INDEPENDENCE,Independence Municipal,Expired Plates,[]
freq,1,148,95,244,6419,144,542,2846,346,9302,476,476,687,9302


In [137]:
cases_df.to_csv("cases.csv", index=False)

In [138]:
case  = get_single_case("704214538")

In [139]:
case.model_dump_json()

'{"case_id":"704214538","court_id":"SMPDB0005_CT44","participants":["InbqrNIkfNR7Xeh5vZcA","yg4GNTLwK0e9pcRIv4Ee"],"related_cases":null,"protection_order":false,"parties":[{"other_attorneylist":[],"pred_code":"ZZ","desc":"Defendant","pty_seqno":2,"birth_date":"09/16/2000","lit_ind":"Y","mid_initial":"EDWIN","pidm":90154411,"addr_line1":"101 JURY LN","addr_city":"BRANSON","birth_datecode":"Y","desc_code":"DFT","formatted_partyaddress":"101 JURY LN\\nBRANSON, MO 65616","last_name":"DEHART","addr_statcode":"MO","co_attorneylist":[],"formatted_telephone":"","attorney_list":[],"addr_atyp":"MA","formatted_birthdate":"2000","prosecuting_atty":false,"criminal_ind":"N","sort_seq":4,"addr_seqno":1,"assoc":0,"addr_zip":"65616","first_name":"DORIAN","party_type":"party","formatted_partyname":"DEHART, DORIAN EDWIN","criminal_case":false},{"addr_statcode":"MO","other_attorneylist":[],"co_attorneylist":[],"pred_code":"ZZ","desc":"Prosecuting Attorney","addr_line2":"COURTHOUSE 2ND FLOOR","pty_seqno":1